In [1]:
import numpy as np
import pandas as pd
import glob
import os
from omegaconf import OmegaConf
import skimage
from tqdm.notebook import tqdm, trange

In [2]:
# Define parameters here
start_date = "2024-01-00"
max_images = 250
output_path = "../outputs"
target_path = "../data/raw/train_kelp/"

In [3]:
# Search for suitable output folders
folders = []
confs = dict()
for day_folder in glob.glob(os.path.join(output_path, "*/")):
    if day_folder.split(os.sep)[-2] < start_date:
        continue       
    for timestamp_folder in glob.glob(os.path.join(day_folder, "*/")):
        if not os.path.exists(os.path.join(timestamp_folder, "preds")):
            continue
        
        # read the .hydra/config.yaml file with omegaconf into a dictionary
        with open(os.path.join(timestamp_folder, ".hydra", "config.yaml"), "r") as f:
            cfg = OmegaConf.load(f)
            
        if "test_size" not in cfg or cfg.test_size != 0.2:
            continue
        
        confs[timestamp_folder] = cfg            
        print(timestamp_folder)
        folders.append(timestamp_folder)

../outputs\2024-01-26\16-14-25\
../outputs\2024-01-26\17-26-33\
../outputs\2024-01-26\17-26-48\
../outputs\2024-01-26\21-26-38\
../outputs\2024-01-26\21-47-29\
../outputs\2024-01-26\23-17-01\
../outputs\2024-01-27\11-47-50\
../outputs\2024-01-27\12-26-05\
../outputs\2024-01-27\12-26-45\
../outputs\2024-01-27\16-07-29\
../outputs\2024-01-27\16-52-01\
../outputs\2024-01-27\18-12-03\
../outputs\2024-01-27\19-24-34\
../outputs\2024-02-14\09-43-04\
../outputs\2024-02-14\14-26-02\
../outputs\2024-02-14\14-45-28\


In [4]:
# Read the filenames inside the first folder
filenames = glob.glob(os.path.join(folders[0], "preds", "*.tif"))
image_names = [os.path.basename(f)[:-9] for f in filenames]

In [5]:
# Create a name based on the config
names = dict()
for folder in folders:
    conf = confs[folder]
    name = conf.model.model_loop_pipeline.model_blocks_pipeline.model_blocks[0].model.model._target_.split(".")[-1]
    if name == "Unet":
        name = f"{conf.model.model_loop_pipeline.model_blocks_pipeline.model_blocks[0].model.model.encoder_name}-{name}"
    name += f"-f{len(conf.model.model_loop_pipeline.pretrain_pipeline.pretrain_steps[1].columns)}"
    name += f"-bs{conf.model.model_loop_pipeline.model_blocks_pipeline.model_blocks[0].batch_size}"
    name += f"-e{conf.model.model_loop_pipeline.model_blocks_pipeline.model_blocks[0].epochs}"
    name += f"-{conf.model.model_loop_pipeline.model_blocks_pipeline.model_blocks[0].criterion._target_.split('.')[-1]}"
    names[folder] = name
for folder, name in names.items():
    print(folder, name)

../outputs\2024-01-26\16-14-25\ vgg11-Unet-f7-bs16-e50-KelpWeightedBatchLoss
../outputs\2024-01-26\17-26-33\ vgg11-Unet-f3-bs16-e50-KelpWeightedBatchLoss
../outputs\2024-01-26\17-26-48\ vgg11-Unet-f3-bs16-e50-KelpWeightedBatchLoss
../outputs\2024-01-26\21-26-38\ resnext50_32x4d-Unet-f3-bs16-e50-KelpWeightedBatchLoss
../outputs\2024-01-26\21-47-29\ vgg11-Unet-f3-bs16-e50-KelpWeightedBatchLoss
../outputs\2024-01-26\23-17-01\ resnext50_32x4d-Unet-f3-bs16-e50-KelpWeightedBatchLoss
../outputs\2024-01-27\11-47-50\ vgg11-Unet-f3-bs16-e50-KelpWeightedBatchLoss
../outputs\2024-01-27\12-26-05\ vgg11-Unet-f3-bs16-e50-KelpWeightedBatchLoss
../outputs\2024-01-27\12-26-45\ vgg11-Unet-f3-bs16-e50-DiceLoss
../outputs\2024-01-27\16-07-29\ vgg11-Unet-f3-bs64-e50-DiceLoss
../outputs\2024-01-27\16-52-01\ vgg11-Unet-f3-bs16-e50-DiceLoss
../outputs\2024-01-27\18-12-03\ vgg11-Unet-f3-bs16-e50-DiceLoss
../outputs\2024-01-27\19-24-34\ vgg11-Unet-f3-bs16-e75-DiceLoss
../outputs\2024-02-14\09-43-04\ SwinUNETR-f1

In [6]:
# Take the first max_images images
image_selection = image_names[:min(max_images, len(image_names))]

In [7]:
# Load the target tiffs into an array
targets = []
for image_name in tqdm(image_selection):
    target = skimage.io.imread(os.path.join(target_path, f"{image_name}_kelp.tif"))
    targets.append(target)
targets = np.array(targets).flatten()

  0%|          | 0/250 [00:00<?, ?it/s]

In [ ]:
# Load the predictions into an array
predictions = dict()
for folder in tqdm(folders):
    preds = []
    for image_name in image_selection:
        pred = skimage.io.imread(os.path.join(folder, "preds", f"{image_name}_pred.tif"))
        preds.append(pred)
    predictions[folder] = np.array(preds).flatten()

  0%|          | 0/16 [00:00<?, ?it/s]

In [ ]:
# Calculate the dice scores and square error
dice_scores = dict()
square_errors = dict()
for folder, preds in tqdm(predictions.items()):
    preds = preds > 0.5
    dice_scores[folder] = 2 * np.sum(targets * preds) / (np.sum(targets) + np.sum(preds))   
    square_errors[folder] = np.mean((targets - preds) ** 2)

In [ ]:
# Remove all models with dice score below 0.6
folders = [folder for folder in folders if dice_scores[folder] > 0.6]

In [ ]:
# Sort the folders by dice score
folders = sorted(folders, key=lambda folder: dice_scores[folder], reverse=True)

In [ ]:
# Create a dataframe with the results
df = pd.DataFrame({
    "name": [names[folder] for folder in folders],
    "dice_score": [dice_scores[folder] for folder in folders],
    "square_error": [square_errors[folder] for folder in folders]
})

In [ ]:
# Create a scatterplot with plotly express
import plotly.express as px
fig = px.scatter(df, x="dice_score", y="square_error", text="name")
fig.show()

In [ ]:
# Step 1 of the algorithm, compute error, error mean per base model, and a covariance matrix
errors = np.array([predictions[folder] - targets for folder in folders])
error_means = np.mean(errors, axis=1)
covariance_matrix = np.cov(errors, bias=True)

In [ ]:
# Step 2. Define a function to estimate the squared error of the ensemble
def ensemble_error(selection, error_means, covariance_matrix):
    selection = np.array(selection) > 0.5
    return np.mean(error_means[selection])**2 + np.mean(covariance_matrix[selection][:, selection])    

In [ ]:
# Compute the actual error for comparison
def true_error(selection, errors):
    selection = np.array(selection) > 0.5
    return np.mean(np.mean(errors[selection],axis=0)**2)

In [ ]:
# Use timeit to compare the two methods
import timeit
print(f"{timeit.timeit(lambda: ensemble_error(np.ones(len(folders)), error_means, covariance_matrix), number=5):.5}s")
print(f"{timeit.timeit(lambda: true_error(np.ones(len(folders)), errors), number=5):.5}s")

In [ ]:
# Step 3. Go through all possible combinations of models and find the best one
limit = np.inf 
best_error = np.inf
best_selection = None
for i in trange(1,min(2**len(folders),limit)):
    selection = [int(x) for x in bin(i)[2:].zfill(len(folders))]
    err = ensemble_error(selection, error_means, covariance_matrix)
    if err < best_error:
        best_error = err
        best_selection = selection
print(best_error)
print(best_selection)
print("Confirmation:", true_error(best_selection, errors))

In [ ]:
simple_selection = np.array([1,1,1,1,0,0,0,0,0,0,0,0,0,0])
print(f"Error (lower is better): Optimised: {best_error:.5f}, simple ensemble: {true_error(simple_selection, errors):.5f}, best model: {true_error([1]+[0]*(len(folders)-1), errors):.5f}")

# compute the dice score for the best and the simple selection ensembles
best_preds = np.mean([predictions[folders[i]] for i in range(len(folders)) if best_selection[i] > 0.5], axis=0) > 0.5
simple_preds = np.mean([predictions[folders[i]] for i in range(len(folders)) if simple_selection[i] > 0.5], axis=0) > 0.5
best_dice = 2 * np.sum(targets * best_preds) / (np.sum(targets) + np.sum(best_preds))
simple_dice = 2 * np.sum(targets * simple_preds) / (np.sum(targets) + np.sum(simple_preds))
print(f"Dice (higher is better): Optimised: {best_dice:.5f}, simple ensemble: {simple_dice:.5f}, best model: {dice_scores[folders[0]]:.5f}")

In [ ]:
# Use the correlation matrix to visualise the variety in the models
corr = np.corrcoef(errors)

# Treat the correlation as a distance and use hierarchical clustering to group the models
import scipy.cluster.hierarchy as sch
import matplotlib.pyplot as plt
dendrogram = sch.dendrogram(sch.linkage(1 - corr, method='ward'), labels=[f"({dice_scores[folder]:.4f}) {names[folder]}" for folder in folders], orientation="left")
# add the names of the models to the x-axis
plt.show()

In [ ]:
# Use UMAP to visualise the models in 2D
import umap
reducer = umap.UMAP()
embedding = reducer.fit_transform(errors.T)
plt.scatter(embedding[:, 0], embedding[:, 1])
for i, txt in enumerate([f"({dice_scores[folder]:.4f}) {names[folder]}" for folder in folders]):
    plt.annotate(txt, (embedding[i, 0], embedding[i, 1]))
plt.show()